In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
data = json.load(open('scraped_players.json'))
# Pick up from our scraped data


In [13]:
# Here's an example row. Look at the stats property... it's a nested dictionary

data_for_df =[]
for player in data:
    for key, val in player.get('stats', {}).get('career_stats', {}).items():
                stat_type = key.lower().split()[0]
                name = player.get('Full Name')
                if not name:
                    continue
                num = player.get('#')
                if not num:
                    continue
                for stat in val:
                    stat['name'] = name
                    stat['num'] = num
                    stat['stat_type'] = stat_type
                    data_for_df.append(stat)


In [ ]:

# We need to recreate tables from our data
# We will create dataframes for each permutation of the data
# So we currently have nested data like this:
#
# player (name, position, etc):
#    -> stats
#        -> career_stats
#            -> stats type (hitting / pitching)
#                -> stats for year 1
#                -> stats for year 2
#                -> ...

# We are going to denormalize "grouped" relationship by unrolling them and repeating each level of data
# So our example above becomes
# name, hitting, year 1, stats...
# name, hitting, year 2, stats...
# name, pitching, year 1, stats...
# name, pitching, year 2, stats...
# etc

In [14]:
# We feed our list of dictionaries into the dataframe class to instantiate a new dataframe
df = pd.DataFrame(data_for_df)


In [15]:

# Attempt to convert everything to a numeric value and by specifying
# errors='ignore' the original value will remain in place if it's not numeric
# Note that this returns a new dataframe so we have to assign it back to our df variable
df = df.apply(pd.to_numeric, errors='ignore')


In [16]:
df.drop('avg', axis=1, inplace=True)

# Drop the avg stat since it's not always present
# Note this time we can use inplace=True and it does NOT return a new dataframe

In [17]:
# Generate the batting average and era for all records
# Note again how we have single arithmetic operators
# but our data is a list- this, again, is the power and short cut of pandas / numpy
df['ba'] = df.h / df.ab
df['era2'] = df.r / df.ip


In [19]:
by_player = df.groupby(('name', 'stat_type'))
by_year = df.groupby('year')
by_type = df.groupby('stat_type')

In [20]:
by_type.get_group('hitting').groupby('year').mean().ba


year
2015    0.210526
2016    0.196937
2017         inf
2018         inf
Name: ba, dtype: float64

In [21]:
by_player.mean().h / by_player.mean().ab

name                  stat_type
Angel Zarate          hitting      0.000000
Ashton McGee          hitting      0.250000
Austin Bergner        hitting           NaN
                      pitching          NaN
Ben Casparius         hitting      1.000000
                      pitching          NaN
Bo Weiss              hitting           NaN
                      pitching          NaN
Brandon Martorano     hitting      0.153846
Brandon Riley         hitting      0.305882
Brendan Illies        hitting      1.000000
Brett Daniels         hitting           NaN
                      pitching          NaN
Caden O'Brien         hitting           NaN
                      pitching          NaN
Clemente Inclan       hitting      0.500000
Cody Roberts          hitting      0.220779
                      pitching          NaN
Cooper Criswell       hitting           NaN
                      pitching          NaN
Dallas Tessar         hitting      0.200000
Dylan Enwiller        hitting      0.000000


In [23]:
df.loc[:, ('name', 'year', 'ba')]

,name,year,ba
0,Brandon Riley,2016,0.592593
1,Brandon Riley,2017,0.195652
2,Brandon Riley,2018,0.083333
3,Kyle Datres,2016,0.153846
4,Kyle Datres,2017,0.270833
5,Kyle Datres,2018,0.307692
6,Brandon Martorano,2017,0.000000
7,Brandon Martorano,2018,0.250000
8,Ashton McGee,2017,0.222222
9,Ashton McGee,2018,0.428571


In [31]:
df.head(30)

,2b,3b,a,ab,app,bavg,bb,bf,bk,cg,...,sho,slg%,so,stat_type,sv,w,wp,year,ba,era2
0,3,1,1.0,27.0,NaN,NaN,1,NaN,NaN,NaN,...,NaN,20.0,3,hitting,NaN,NaN,NaN,2016,0.592593,NaN
1,5,7,4.0,46.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,37.0,6,hitting,NaN,NaN,NaN,2017,0.195652,NaN
2,0,2,1.0,12.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,16.0,0,hitting,NaN,NaN,NaN,2018,0.083333,NaN
3,2,3,8.0,39.0,NaN,NaN,8,NaN,NaN,NaN,...,NaN,27.0,4,hitting,NaN,NaN,NaN,2016,0.153846,NaN
4,1,7,6.0,48.0,NaN,NaN,20,NaN,NaN,NaN,...,NaN,29.0,6,hitting,NaN,NaN,NaN,2017,0.270833,NaN
5,0,1,1.0,13.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,8.0,2,hitting,NaN,NaN,NaN,2018,0.307692,NaN
6,0,0,0.0,5.0,NaN,NaN,2,NaN,NaN,NaN,...,NaN,6.0,3,hitting,NaN,NaN,NaN,2017,0.000000,NaN
7,0,5,0.0,8.0,NaN,NaN,1,NaN,NaN,NaN,...,NaN,5.0,0,hitting,NaN,NaN,NaN,2018,0.250000,NaN
8,2,7,2.0,45.0,NaN,NaN,2,NaN,NaN,NaN,...,NaN,36.0,4,hitting,NaN,NaN,NaN,2017,0.222222,NaN
9,0,0,0.0,7.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,8.0,0,hitting,NaN,NaN,NaN,2018,0.428571,NaN


In [29]:
cols = df.columns.tolist()

In [30]:
df["name"]

0            Brandon Riley
1            Brandon Riley
2            Brandon Riley
3              Kyle Datres
4              Kyle Datres
5              Kyle Datres
6        Brandon Martorano
7        Brandon Martorano
8             Ashton McGee
9             Ashton McGee
10          Dylan Enwiller
11           Dallas Tessar
12             Ike Freeman
13             Ike Freeman
14            Zack Gahagan
15            Zack Gahagan
16            Zack Gahagan
17            Zack Gahagan
18            Cody Roberts
19            Cody Roberts
20            Cody Roberts
21            Cody Roberts
22            Cody Roberts
23            Cody Roberts
24           Josh Ladowski
25           Josh Ladowski
26           Josh Ladowski
27           Michael Busch
28           Michael Busch
29             Earl Semper
              ...         
69              Josh Hiatt
70           Caden O'Brien
71           Caden O'Brien
72        Joey Lancellotti
73        Joey Lancellotti
74                Bo Weiss
7

In [32]:
indices = [i for i, s in enumerate(cols) if 'name' in s]

In [34]:
cols = cols[22:] + cols[:22]

In [35]:
indic = [i for i, s in enumerate(cols) if 'year' in s]

In [36]:
cols.insert(1, cols.pop(18))

In [40]:
cols

['num',
 'ba',
 'ob%',
 'po',
 'r',
 'rbi',
 'sb',
 'season',
 'sf',
 'sfa',
 'sh',
 'sho',
 'slg%',
 'so',
 'stat_type',
 'sv',
 'w',
 'wp',
 'year',
 'era2',
 '2b',
 '3b',
 'a',
 'ab',
 'app',
 'bavg',
 'bb',
 'bf',
 'bk',
 'cg',
 'e',
 'er',
 'era',
 'gdp',
 'gp',
 'gs',
 'h',
 'hbp',
 'hr',
 'ip',
 'l',
 'name']